<a href="https://colab.research.google.com/github/Beluuuuuuga/python_machine_learning_2nd_edition/blob/master/kaggle_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Google driveにファイル保存する用
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# 4/wwEVHBbLp_T5iqhskY-swvFHQ_O_x3PvyYcgC8mQkXYW4H14lP9mwa0

In [3]:
# kaggle API使用
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!ls ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
kaggle.json


In [53]:
# 使用を確認
!kaggle competitions list

ref                                                                deadline             category             reward  teamCount  userHasEntered  
-----------------------------------------------------------------  -------------------  ---------------  ----------  ---------  --------------  
digit-recognizer                                                   2030-01-01 00:00:00  Getting Started   Knowledge       2248            True  
titanic                                                            2030-01-01 00:00:00  Getting Started   Knowledge      16628            True  
house-prices-advanced-regression-techniques                        2030-01-01 00:00:00  Getting Started   Knowledge       4576           False  
connectx                                                           2030-01-01 00:00:00  Getting Started   Knowledge        345           False  
competitive-data-science-predict-future-sales                      2020-12-31 23:59:00  Playground            Kudos       5957    

In [5]:
# titanicのデータをダウンロード、アンジップ
!kaggle competitions download -c titanic

  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 22.3MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 3.08MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 24.5MB/s]


In [0]:
import pandas as pd
import numpy as np

In [0]:
train =  pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample_submission = pd.read_csv('gender_submission.csv')

## カレーちゃんチュートリアル

In [47]:
# https://www.kaggle.com/currypurin/titanic-lightgbm
# SexとEmbarkedのOne-Hotエンコーディング
train = pd.get_dummies(train, columns=['Sex', 'Embarked'])
test = pd.get_dummies(test, columns=['Sex', 'Embarked'])
 # 補足 : Sexは2値なので、One-Hotエンコーディングは不要(してもいいが普通はしない)ですが、今回は行なっています

# 不要な列の削除
train.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)
test.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

# trainの表示
display(train.head())
X_train = train.drop(['Survived'], axis=1)  # X_trainはtrainのSurvived列以外
Y_train = train['Survived']  # Y_trainはtrainのSurvived列

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.0,1,0,7.2500,0,1,0,0,1
1,1,1,38.0,1,0,71.2833,1,0,1,0,0
2,1,3,26.0,0,0,7.9250,1,0,0,0,1
3,1,1,35.0,1,0,53.1000,1,0,0,0,1
4,0,3,35.0,0,0,8.0500,0,1,0,0,1


In [48]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# X_trainとY_trainをtrainとvalidに分割
train_x, valid_x, train_y, valid_y = train_test_split(X_train, Y_train, test_size=0.33, random_state=0)

# LightGBMの分類器をインスタンス化
gbm = lgb.LGBMClassifier(objective='binary')

# trainとvalidを指定し学習
gbm.fit(train_x, train_y, eval_set = [(valid_x, valid_y)],
        early_stopping_rounds=20,  # 20回連続でlossが下がらなかったら終了
        verbose=10  # 10round毎に、lossを表示
) ;

Training until validation scores don't improve for 20 rounds.
[10]	valid_0's binary_logloss: 0.465153
[20]	valid_0's binary_logloss: 0.419539
[30]	valid_0's binary_logloss: 0.40614
[40]	valid_0's binary_logloss: 0.402566
[50]	valid_0's binary_logloss: 0.409918
[60]	valid_0's binary_logloss: 0.415006
Early stopping, best iteration is:
[40]	valid_0's binary_logloss: 0.402566


In [49]:
# valid_xについて推論
oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)  # oofはout of fold
print('score', round(accuracy_score(valid_y, oof)*100,2), '%')  # 正解率の表示

# out of foldは学習に使っているデータ以外の予測

score 81.69 %


In [0]:
# testの予測
test_pred = gbm.predict(test, num_iteration=gbm.best_iteration_)  # testの予測
sample_submission['Survived'] = test_pred  # sample_submissionのSurvived列をtest_predに置き換え
sample_submission.to_csv('train_test_split.csv', index=False)  # csvファイルの書き出し

In [51]:
# 3分割交差検証を指定し、インスタンス化
from sklearn.model_selection import KFold
kf = KFold(n_splits=3)  # 3分割交差検証のためにインスタンス化

# スコアとモデルを格納するリスト
score_list = []
models = []

for fold_, (train_index, valid_index) in enumerate(kf.split(X_train, Y_train)):
    train_x = X_train.iloc[train_index]
    valid_x = X_train.iloc[valid_index]
    train_y = Y_train[train_index]
    valid_y = Y_train[valid_index]
    
    print(f'fold{fold_ + 1} start')

    gbm = lgb.LGBMClassifier(objective='binary')
    gbm.fit(train_x, train_y, eval_set = [(valid_x, valid_y)],
                early_stopping_rounds=20,
                verbose= -1) # 学習の状況を表示しない
    
    oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
    score_list.append(round(accuracy_score(valid_y, oof)*100,2))
    models.append(gbm)  # 学習が終わったモデルをリストに入れておく
    print(f'fold{fold_ + 1} end\n' )
print(score_list, '平均score', np.mean(score_list), "%")  

fold1 start
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[21]	valid_0's binary_logloss: 0.468911
fold1 end

fold2 start
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[21]	valid_0's binary_logloss: 0.427376
fold2 end

fold3 start
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[40]	valid_0's binary_logloss: 0.382065
fold3 end

[81.48, 82.49, 84.18] 平均score 82.71666666666667 %


In [0]:
# testの予測
test_pred = np.zeros((len(test), 3))  # 行:len(test), 列:3のall zeroの配列を用意

for fold_, gbm in enumerate(models):  # 学習ずみのmodelをgbmに入れる
    pred_ = gbm.predict(test, num_iteration=gbm.best_iteration_)  # testの予測
    test_pred[:, fold_] = pred_  # １回目は0列目、2回目は1列目、2回目は3列目に格納

pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)  # 平均をとって、0と１に変換
sample_submission['Survived'] = pred
sample_submission.to_csv('3-fold_cross-validation.csv',index = False)

In [56]:
# submitする時
!kaggle competitions submit -c titanic -f 3-fold_cross-validation.csv -m "first submit"

100% 2.77k/2.77k [00:00<00:00, 9.02kB/s]
Successfully submitted to Titanic: Machine Learning from Disaster

In [57]:
# GridSearchCVをimport
from sklearn.model_selection import GridSearchCV

gbm = lgb.LGBMClassifier(objective='binary')

# 試行するパラメータを羅列する
params = {
    'max_depth': [2, 3, 4, 5],
    'reg_alpha': [0, 1, 10, 100],
    'reg_lambda': [0, 1, 10, 100],
}

grid_search = GridSearchCV(
                           gbm,  # 分類器を渡す
                           param_grid=params,  # 試行してほしいパラメータを渡す
                           cv=3,  # 3分割交差検証でスコアを確認
                          )

grid_search.fit(X_train, Y_train)  # データを渡す

print(grid_search.best_score_)  # ベストスコアを表示
print(grid_search.best_params_)  # ベストスコアのパラメータを表示

0.8271604938271605
{'max_depth': 3, 'reg_alpha': 1, 'reg_lambda': 0}


In [58]:
# スコアとモデルを格納するリスト
score_list = []
test_pred = np.zeros((len(test), 3))


for fold_, (train_index, valid_index) in enumerate(kf.split(X_train, Y_train)):
    train_x = X_train.iloc[train_index]
    valid_x = X_train.iloc[valid_index]
    train_y = Y_train[train_index]
    valid_y = Y_train[valid_index]
    
    print(f'fold{fold_ + 1} start')

    gbm = lgb.LGBMClassifier(objective='binary', max_depth=3, reg_alpha=1,
                             reg_lambda=0)
    gbm.fit(train_x, train_y,
            eval_set = [(valid_x, valid_y)],
            early_stopping_rounds=20,
            verbose= -1)
    
    oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
    score_list.append(round(accuracy_score(valid_y, oof)*100,2))
    test_pred[:, fold_] = gbm.predict(test, num_iteration=gbm.best_iteration_)
    print(f'fold{fold_ + 1} end\n' )
print(score_list, '平均score', np.mean(score_list))
pred = (np.mean(test_pred, axis=1) > 0.5).astype(int)
sample_submission['Survived'] = pred
sample_submission.to_csv('glid_search.csv', index=False)  # scoreは0.77511

fold1 start
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[54]	valid_0's binary_logloss: 0.453196
fold1 end

fold2 start
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[34]	valid_0's binary_logloss: 0.414269
fold2 end

fold3 start
Training until validation scores don't improve for 20 rounds.
Early stopping, best iteration is:
[76]	valid_0's binary_logloss: 0.382864
fold3 end

[79.8, 81.48, 83.84] 平均score 81.70666666666666


In [60]:
# submitする時
!kaggle competitions submit -c titanic -f glid_search.csv -m "first submit"

100% 2.77k/2.77k [00:00<00:00, 10.7kB/s]
Successfully submitted to Titanic: Machine Learning from Disaster